# Download buildings in France

In [28]:
import os
import requests
import subprocess
import py7zr
import geopandas as gpd

In [8]:
# Base URL
BASE_URL = "https://data.geopf.fr/telechargement/download/BDTOPO/BDTOPO_3-5_TOUSTHEMES_GPKG_WGS84G_FRA_2025-06-15"

# All parts
PARTS = [f"{BASE_URL}/BDTOPO_3-5_TOUSTHEMES_GPKG_WGS84G_FRA_2025-06-15.7z.{i:03d}" for i in range(1, 10)]

# Local dirs
DOWNLOAD_DIR = "/data/uscuni-ulce/extension/france/download"
EXTRACT_DIR = "/data/uscuni-ulce/extension/france/extracted"

Download archive.

In [ ]:
# Step 1: Download all parts with wget (resumable)
for url in PARTS:
    fname = os.path.join(DOWNLOAD_DIR, os.path.basename(url))
    if os.path.exists(fname):
        print(f"✅ Already downloaded: {fname}")
    print(f"⬇️ Downloading {url}")
    subprocess.run(["wget", "-c", "-O", fname, url], check=True)


Concatenate.

In [18]:
PARTS = [
    "BDTOPO_3-5_TOUSTHEMES_GPKG_WGS84G_FRA_2025-06-15.7z.001",
    "BDTOPO_3-5_TOUSTHEMES_GPKG_WGS84G_FRA_2025-06-15.7z.002",
    "BDTOPO_3-5_TOUSTHEMES_GPKG_WGS84G_FRA_2025-06-15.7z.003",
    "BDTOPO_3-5_TOUSTHEMES_GPKG_WGS84G_FRA_2025-06-15.7z.004",
    "BDTOPO_3-5_TOUSTHEMES_GPKG_WGS84G_FRA_2025-06-15.7z.005",
    "BDTOPO_3-5_TOUSTHEMES_GPKG_WGS84G_FRA_2025-06-15.7z.006",
    "BDTOPO_3-5_TOUSTHEMES_GPKG_WGS84G_FRA_2025-06-15.7z.007",
    "BDTOPO_3-5_TOUSTHEMES_GPKG_WGS84G_FRA_2025-06-15.7z.008",
    "BDTOPO_3-5_TOUSTHEMES_GPKG_WGS84G_FRA_2025-06-15.7z.009",
]

COMBINED_ARCHIVE = os.path.join(DOWNLOAD_DIR, "full_france.7z")

# -------------------------
# Step 1: Concatenate parts
# -------------------------
print("🔗 Concatenating parts...")
with open(COMBINED_ARCHIVE, "wb") as wfd:
    for part in PARTS:
        part_path = os.path.join(DOWNLOAD_DIR, part)
        print(f"  Adding {part_path}")
        with open(part_path, "rb") as fd:
            while True:
                chunk = fd.read(10_000_000)  # 10 MB chunks
                if not chunk:
                    break
                wfd.write(chunk)
print(f"✅ Combined archive saved as: {COMBINED_ARCHIVE}")

🔗 Concatenating parts...
  Adding /data/uscuni-ulce/extension/france/download/BDTOPO_3-5_TOUSTHEMES_GPKG_WGS84G_FRA_2025-06-15.7z.001
  Adding /data/uscuni-ulce/extension/france/download/BDTOPO_3-5_TOUSTHEMES_GPKG_WGS84G_FRA_2025-06-15.7z.002
  Adding /data/uscuni-ulce/extension/france/download/BDTOPO_3-5_TOUSTHEMES_GPKG_WGS84G_FRA_2025-06-15.7z.003
  Adding /data/uscuni-ulce/extension/france/download/BDTOPO_3-5_TOUSTHEMES_GPKG_WGS84G_FRA_2025-06-15.7z.004
  Adding /data/uscuni-ulce/extension/france/download/BDTOPO_3-5_TOUSTHEMES_GPKG_WGS84G_FRA_2025-06-15.7z.005
  Adding /data/uscuni-ulce/extension/france/download/BDTOPO_3-5_TOUSTHEMES_GPKG_WGS84G_FRA_2025-06-15.7z.006
  Adding /data/uscuni-ulce/extension/france/download/BDTOPO_3-5_TOUSTHEMES_GPKG_WGS84G_FRA_2025-06-15.7z.007
  Adding /data/uscuni-ulce/extension/france/download/BDTOPO_3-5_TOUSTHEMES_GPKG_WGS84G_FRA_2025-06-15.7z.008
  Adding /data/uscuni-ulce/extension/france/download/BDTOPO_3-5_TOUSTHEMES_GPKG_WGS84G_FRA_2025-06-15.7

Extract.

In [19]:
print(f"📦 Extracting {COMBINED_ARCHIVE} ...")
os.makedirs(EXTRACT_DIR, exist_ok=True)
with py7zr.SevenZipFile(COMBINED_ARCHIVE, mode="r") as archive:
    archive.extractall(path=EXTRACT_DIR)

print(f"🎉 Extracted to: {EXTRACT_DIR}")

📦 Extracting /data/uscuni-ulce/extension/france/download/full_france.7z ...
🎉 Extracted to: /data/uscuni-ulce/extension/france/extracted


In [25]:
!ls /data/uscuni-ulce/extension/france/extracted/BDTOPO_3-5_TOUSTHEMES_GPKG_WGS84G_FRA_2025-06-15/BDTOPO/1_DONNEES_LIVRAISON_2025-06-00371/BDT_3-5_GPKG_WGS84G_FRA-ED2025-06-15/BATI/batiment.gpkg

batiment.gpkg		       construction_surfacique.gpkg
batiment_rnb_lien_bdtopo.gpkg  ligne_orographique.gpkg
cimetiere.gpkg		       pylone.gpkg
construction_lineaire.gpkg     reservoir.gpkg
construction_ponctuelle.gpkg   terrain_de_sport.gpkg


In [ ]:
bati = gpd.read_file("/data/uscuni-ulce/extension/france/extracted/BDTOPO_3-5_TOUSTHEMES_GPKG_WGS84G_FRA_2025-06-15/BDTOPO/1_DONNEES_LIVRAISON_2025-06-00371/BDT_3-5_GPKG_WGS84G_FRA-ED2025-06-15/BATI/batiment.gpkg")
bati